The initial method for processing exam data involved a four-step process:

1. Extracting exams using ChatGPT prompts.
2. Grouping the extracted exams through clustering.
3. Mapping the clusters to specific skills.
4. Assigning skills to actual courses.
However, this approach was ultimately rejected due to bad results. 

Nevertheless, the clusters generated in this process can still be utilized in Notebook 6. While it is possible to run Notebook 4 without the clustering step, I've observed that incorporating the results of clustering leads to better outcomes.

In [ ]:
# This is notebook for clustering exams and mapping those clusters to skills
# import libraries
import pandas as pd
import ast
import time
from transformers import pipeline
model_ckpt = "papluca/xlm-roberta-base-language-detection"
pipe = pipeline("text-classification", model=model_ckpt) # model which detects language
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import re
import pandas as pd
import os
import openai

# STEP 1 extract exams


In [198]:
#use your key and organization credentials. If you do not have free credit from openai anymore, it is not for free.
key = "PUT-YOUR-KEY-HERE"
openai.organization = "PUT-YOUR-ORG-HERE"
openai.api_key = key

In [ ]:

#  Approach can be modified in the following ways: 1.can adjust prompt as required
# 2. use another chatbot (alternative is given in  notebook 6) 
# 3. or can use classification model as in notebook 03-b
# the ultimate result is a list of exams which are identified from the column Assessment details
def ask_chatgpt(question):
    response = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',
        n=1,
        messages=[
            {"role": "system", "content": "You are an algorithm which extracts assessment type from the text. Types are comma separated, packed in square brackets, Example answer: [written exam, essay]. If information is not given just text: [None]. No additional words allowed."},
            {"role": "user", "content": question},
        ])

    message = response.choices[0]['message']
    return message['content']

# running the code to extract exam skills
for year in range(2007, 2023):
    for period in range(4, 6):
        ind = str(year)+("00")+str(period)
        df = pd.read_csv('all_data_merged/student_courses_%s.csv' %ind) #fixme
        
        if "Assessment details" in df.columns: 
            for idx, row in df.iterrows():
                try:
                    if type(df.loc[idx, "Assessment details"]) != float and df.loc[idx, "new"]:
                       smth = []
                       s = ask_chatgpt(row["Assessment details"])
                       lst = re.findall(r'[^\[\],]+', s)
                       result = [item.strip() for item in lst if item.strip()]
                       smth.extend(result)
                       print(idx, str(smth))
                       df.loc[idx, 'exam_types_processed'] = str(smth)
                       time.sleep(20) # essential to put delay to avoid reaching limit per min (at the time of writing it was 3 prompts per min)
                except Exception as e:
                    print(e, idx, ind)
            df.to_csv('all_data_merged/exam_info_processed_%s.csv' % ind,  encoding='utf-8', index=False)  # fixme


In [ ]:
#small explanation on how the output of chatgpt is parsed
s = "[ mündliche Prüfung, Essay ]"
lst = re.findall(r'[^\[\],]+', s)
result = [item.strip() for item in lst if item.strip()]
print(result)  


In [53]:
res ....
res.to_csv('exam_types.csv',  encoding='utf-8', index=False)  


# STEP 2 group exams

In [ ]:
# first the exam types are translated, becauase all the pretrained models perform much better in english

from transformers import pipeline

model_name = "Helsinki-NLP/opus-mt-de-en"  # Translation model for German to English
translator = pipeline("translation", model=model_name, tokenizer=model_name)


In [32]:
res = pd.read_csv('exam_types.csv')   # read the distinct exam types from prev step
res.set_index("0",inplace=True)
dic = res.to_dict()["1"]

In [35]:
# extract english and german exams
en_exams = []
de_exams = []
rest_exams = []
for key, value in dic.items():
    if value:
        lang = pipe(key)[0]["label"]
        if lang == "en":
                en_exams.append(key)
        elif lang == "de": 
            de_exams.append(key)
        else:
            print(lang, key)
            rest_exams.append(key)

nl experimentaldesign
fr lesson protocol
tr inputreferat
ur protocol
it colloquium
fr exercices
hi organic synthesis
ur test
hi laboratory exam
ur research paper
nl thesenpapier
it perception training
tr projektreferat
es reflexionspaper
ur test paper
sw analog retake
tr podcast/video
ur exposé
nl plenardiskussionen
ur thematischer essay
pt osce exam
fr rechercheportfolio
pt formativer osce
ur input-referat
ur academic literature
ur aktive partizipation
nl practical masterprojekt
ur mcq
fr quellenanalyse
it kritische lektüre
ur praktikum
it projektproposal
ur action learning
ur bibliographie
ur mc-test
ur film analysis
hi lehrvideo
hi testat
hi textkritik
ur ethnographic research
ru gruppenreferate
hi co-referat
hi portfoliobericht
nl referaten
tr diskussion
nl semesterplan
fr recherche
nl ethische argumentation
ur korrektur
it research diary
sw benotung
ur übung
ur immanent examination
ur klausur
ur online-klausur
sw retake
ur written quiz
ur ethnographic input
fr présentation
it pros

In [37]:
# translate the exams
translated = [] 
translated_2 = {}
for el in de_exams:
    translated_text = translator(el, max_length=40)[0]['translation_text']
    print(el, translated_text,  translated_text_1)
    translated.append(translated_text)
    translated_2[el] = translated_text

wöchentliche lektüre weekly reading weekly reading
schriftliche leistungskontrolle written performance control written performance control
anwesenheitspflicht Compulsory status Compulsory status
mitarbeit co-operation co-operation
online-kurse Online courses Online courses
kurzvortrag short presentation short presentation
leistung Performance Performance
forschungskonzept Research concept Research concept
impuls-referat impulse-referat impulse-referat
lektüreberichte Reading reports Reading reports
take-home aufgaben take-home tasks take-home tasks
arbeitsaufträge work orders work orders
seminararbeit Seminar work Seminar work
schriftliche arbeit written work written work
abschlussbericht final report final report
begleitende mc-tests Accompanying mc tests Accompanying mc tests
schriftlicher bericht in writing in writing
seminar-arbeit Seminar work Seminar work
proseminar-arbeit proseminar work proseminar work
diskussionsbeteiligung Discussing participation Discussing participation
tak

kurzpräsentation short presentation short presentation
schriftliche übung written exercise written exercise
präsenzklausur Presence clause Presence clause
mündliche beteiligung Oral participation Oral participation
computerunterstützter test computer assisted test computer assisted test
kurzbiographie Short biography Short biography
wörterblatt Wordsheet Wordsheet
studienteilnahme participation in studies participation in studies
projektergebnisse project results project results
diskussionsleitung Discussion direction Discussion direction
besuch aller vorträge visit of all presentations visit of all presentations
vortrag/referat lecture/referral lecture/referral
abschlusstest Final test Final test
reflexionsarbeit Reflexion work Reflexion work
forschungsprojektplan research project plan research project plan
schriftliche klausur Written clausage Written clausage
osce-prüfung osce test osce test
gruppenarbeit group work group work
schlussprüfung Final verification Final verification
sch

In [8]:
# combine all the translations in english and exams which were originally in english
en_exams = list(en_exams)
en_exams.extend(translated)
en_exams = list(set(en_exams))

In [30]:
# here starts the clustering. The next 3 cells are performing clustering and saving the resulrs in exam_clusters csv

# Initialize TF-IDF vectorizer
ngram_range = (1, 3)  
vectorizer = TfidfVectorizer(ngram_range=ngram_range)


# Vectorize the strings
vectorized_strings = vectorizer.fit_transform(en_exams)
print(vectorized_strings.shape)


# Perform K-means clustering
k = 100 # parameter to tune
kmeans = KMeans(n_clusters=k).fit(vectorized_strings)

# Get the centroid of each cluster
centroids = kmeans.cluster_centers_

# Find the closest exam to each centroid
representative_exams = []
for centroid in centroids:
    closest_exam_idx = np.argmin(np.linalg.norm(vectorized_strings - centroid, axis=1))
    closest_exam = en_exams[closest_exam_idx]
    representative_exams.append(closest_exam)


(778, 1370)


In [ ]:

# Get the cluster labels
cluster_labels = kmeans.labels_
cluster_centroids = kmeans.cluster_centers_

# Create a dictionary to store exams for each cluster
cluster_exams = {i: [] for i in range(k)}
cluster_centroids = {i: [] for i in range(k)}

# Iterate over the skills and cluster labels
for i, exam in enumerate(en_exams):
    cluster = cluster_labels[i]
    cluster_exams[cluster].append(exam)

# Get the centroids for each cluster
centroids = kmeans.cluster_centers_

# Assign the closest exam to each cluster centroid
for cluster, centroid in enumerate(centroids):
    closest_exam_idx = np.argmin(np.linalg.norm(vectorized_strings - centroid, axis=1))
    closest_exam = en_exams[closest_exam_idx]
    cluster_centroids[cluster] = closest_exam
    
english = {}
# Print exams and centroids for each cluster
for cluster, exams in cluster_exams.items():
    print(f"Cluster {cluster+1} exams:")
    for exam in exams:
        print(exam)
        english[exam] = cluster_centroids[cluster]
    print("Centroid:", cluster_centroids[cluster])
    print()


In [232]:
df_english = pd.DataFrame.from_dict(english, orient='index', columns=['Cluster']) # form a datafrane 

# Add the exams as a separate column
df_english['Exam'] = df_english.index

# Set the index name
df_english.index.name = 'Index'

# Save the DataFrame as a CSV file
df_english.to_csv('exam_clusters.csv', index=False) # finally save the results

# afterwards I opened this file in excel and I did some manual postprocessing, afterwards that was saved as exam_clusters_processed_2
# and accessed in notebook 6

In [ ]:
# this is an alternatve clustering method, but Kmean performed a bit better
# Initialize TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Vectorize the strings
vectorized_strings = vectorizer.fit_transform(en_exams)
print(vectorized_strings.shape)

# Perform Agglomerative Clustering
k = 70  # parameter to tune
agg_clustering = AgglomerativeClustering(n_clusters=k).fit(vectorized_strings.toarray())

# Get the cluster labels
cluster_labels = agg_clustering.labels_

# Create a dictionary to store exams for each cluster
cluster_exams = {i: [] for i in range(k)}
cluster_centroids = {i: [] for i in range(k)}

# Iterate over the exams and cluster labels
for i, exam in enumerate(en_exams):
    cluster = cluster_labels[i]
    cluster_exams[cluster].append(exam)

# Assign the closest exam to each cluster centroid
for cluster in range(k):
    cluster_indices = np.where(cluster_labels == cluster)[0]
    cluster_vectors = vectorized_strings[cluster_indices].toarray()
    centroid = np.mean(cluster_vectors, axis=0)
    closest_exam_idx = np.argmin(np.linalg.norm(cluster_vectors - centroid, axis=1))
    closest_exam = en_exams[cluster_indices[closest_exam_idx]]
    cluster_centroids[cluster] = closest_exam

english = {}
# Print exams and centroids for each cluster
for cluster, exams in cluster_exams.items():
    print(f"Cluster {cluster+1} exams:")
    for exam in exams:
        print(exam)
        english[exam] = cluster_centroids[cluster]
    print("Centroid:", cluster_centroids[cluster])
    print() 

# STEP 3 exam to skill map

In [ ]:
# this part is no longer relevant, because we do not just map cluster to skill anymore
# therefore everything below can be ignored

In [ ]:
df = pd.read_csv('exam_clusters_processed.csv')
clusters = list(set(df["Cluster"]))
all_skills = pd.read_csv('skills.csv') # fixme
all_skills = list(set(all_skills["skill"].tolist()))
new = pd.DataFrame(columns=["Cluster", "Skills"])
for cluster in clusters:
    if cluster not in processed:
        skills = []
        print("Cluster:", cluster)
        print("Cluster values:", df[df["Cluster"] == cluster]["Exam"].values)
        print("skills:", all_skills)

        while True:
            skill = input("Enter a skill (or 0 to move to the next cluster): ")

            if skill == '0':
                break

            skills.append(skill)

        new = new.append({"Cluster": cluster, "Skills": skills}, ignore_index=True)
        
new.to_csv('clusters_skill_map.csv', sep=';', index=False)

# STEP 4 final course to skill map

In [53]:
import ast
cl_exam_map = pd.read_csv('exam_clusters_processed.csv')
cl_skill_map = pd.read_csv('clusters_skill_map.csv', sep=';')

In [54]:
cl_exam_map = cl_exam_map.set_index('Exam')['Cluster'].to_dict()

In [55]:
cl_skill_map = cl_skill_map.set_index('Cluster')['Skills'].to_dict()

In [56]:
for key, value in cl_skill_map.items():
    parsed_list = ast.literal_eval(value)
    parsed_list = [string.strip("'") for string in parsed_list]
    cl_skill_map[key] = parsed_list


In [ ]:

for year in range(2007, 2023):
    for period in range(4, 6):
        ind = str(year) + "00" + str(period)
        df = pd.read_csv('../all_data_merged/exam_info_processed_%s.csv' % ind)
        df = df.groupby("Awobjid").apply(lambda x: x.loc[x["exam_types_processed"].str.len().idxmax()])
        df.reset_index(drop=True, inplace=True)
        for idx, row in df.iterrows():
            parsed_list = ast.literal_eval(df.loc[idx, "exam_types_processed"])
            skill_list = []
            for element in parsed_list:
                # translate if necessary
                if element in translated_2:
                    el = translated_2[element]
                else:
                    el = element
                # access correponding cluster and skills
                if el in cl_exam_map and cl_exam_map[el] in cl_skill_map:
                         skill_list.extend(cl_skill_map[cl_exam_map[el]])

            print(skill_list)
            df.loc[idx, "exam_skills"] = str(list(set(skill_list)))
        df.to_csv('exam_info_result_%s.csv' % ind,  encoding='utf-8', index=False)  
